# Parameter sweeps

The `DifferentialEquations.jl` and `JuliaDynamics` ecosystems are useful
for making highly optimized parameter sweeps of dynamical systems.

In particular, we'll look at:
- `OrdinaryDiffEq` (DifferentialEquations)
- `ChaosTools` (JuliaDynamics)

The tools from JuliaDynamics' `DynamicalSystems.jl` (of which `ChaosTools.jl` is
a part) are documented in tutorial format very well in the book by Datseris &
Parlitz.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Dev/siam-student-conf-2025/notebooks`


# Lyapunov spectrum analysis

We'll compute two sweeps:
- *Leading Lyapunov exponent*, which shows parameter regions corresponding to
  chaotic dynamics.
- *Lyapunov dimension*, which shows the fractal dimension of the attractor in the
  state space.

Let's compute these for a single point in parameter space to see how they work.

In [2]:
using OrdinaryDiffEq

In [10]:
function lorenz!(du, u, p, t)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u
  
  # Define the Lorenz system equations.
  du[1] = σ * (y - x)
  du[2] = x * (ρ - z) - y
  du[3] = x * y - β * z

  # Return the derivatives; we will need this for BifurcationKit.jl.
  du
end

lorenz! (generic function with 1 method)

In [9]:
p = [
  28.0,   # ρ
  10.0,   # σ
  8.0/3.0 # β
]

u0 = [
  10.0, # x
  10.0, # y
  10.0  # z
]

tspan = (0.0, 1e6)

(0.0, 1.0e6)

In [10]:
# Integrate.
prob = ODEProblem(lorenz!, u0, tspan, p)
solution = solve(prob, Tsit5(), abstol=1e-7, reltol=1e-7)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 772384-element Vector{Float64}:
     0.0
     0.006003407934143237
     0.010368505831335927
     0.016647770622131776
     0.022830356980822322
     0.03004223932603737
     0.03761790564643813
     0.04590463197518049
     0.05470294590706537
     0.06411465928010099
     ⋮
  9999.941239456757
  9999.948043621418
  9999.954528636945
  9999.961649038387
  9999.96898606132
  9999.976895750453
  9999.985330285655
  9999.994460225584
 10000.0
u: 772384-element Vector{Vector{Float64}}:
 [10.0, 10.0, 10.0]
 [10.029718115640433, 11.004845343821025, 10.467542332641681]
 [10.086735098810065, 11.716707788706875, 10.84229404742576]
 [10.216777759684287, 12.714657411752464, 11.434449812177528]
 [10.395549285802868, 13.667902087852132, 12.081137919893681]
 [10.661304929207754, 14.742231541286515, 12.919409913624254]
 [10.999293332286388, 15.823410028275172, 13.90261017699149]
 [11.428512422238061, 16.941917441413686, 15

In [ ]:
# Plot solution.

using GLMakie

# Extract the solution components.
xs = [u[1] for u in solution.u]
ys = [u[2] for u in solution.u]
zs = [u[3] for u in solution.u]

# Create the 3D plot.
fig = Figure()
ax = Axis3(
  fig[1, 1], 
  xlabel = "x", 
  ylabel = "y", 
  zlabel = "z",
  title = "Lorenz system trajectory"
)

# Plot the solution.
lines!(ax, xs, ys, zs, linewidth = 1)

# Display the figure.
fig

ChaosTools.jl wants the system specified not as an `ODEProblem`, but instead as
a `DynamicalSystem`, which is a type defined by ChaosTools.jl.
The appropriate subtype of `DynamicalSystem` for the Lorenz system is
`CoupledODEs`.

In [ ]:
using ChaosTools

# Set up the system.
lorenz_system = CoupledODEs(
  lorenz!,
  u0,
  p
)

In [56]:
# Compute the Lyapunov spectrum.
λ = lyapunovspectrum(
  lorenz_system, # The DynamicalSystem.
  Int(3e2);      # Integration steps.
  Ttr = 1e2,     # Transient time.
  Δt = 1e-1      # Time between orthonormalization steps.
)

3-element Vector{Float64}:
   0.8976573932058167
   0.006348284187339487
 -14.570599449397237

In [5]:
# Calculate the Lyapunov dimension (Kaplan-Yorke dimension).
# This is defined as k + sum(λs[1:k])/|λs[k+1]|, where k is the largest
# integer such that the sum of the first k Lyapunov exponents is non-negative.
function lyapunov_dimension(λ)
    k = 0
    sum_λ = 0.0
    
    # Find the largest k where the sum is still non-negative.
    for (i, λ) in enumerate(λ)
        sum_λ += λ
        if sum_λ < 0
            break
        end
        k = i
    end
    
    # If all exponents are positive or k is the last index, return k.
    if k == length(λs) || k == 0
        return k
    end
    
    # Calculate the Kaplan-Yorke dimension.
    return k + sum(λ[1:k])/abs(λ[k+1])
end

lyapunov_dimension (generic function with 1 method)

In [59]:
lyap_dim = lyapunov_dimension(λ)
println("Estimated Lyapunov dimension: $lyap_dim")

Estimated Lyapunov dimension: 2.0620431355986906


We should expect a Lyapunov dimension of $2.06 \pm 0.01$.

# Sweeps over $\rho, \sigma$

In [6]:
using StaticArrays

In [18]:
# Define parameter range.
ρs = range(10.0, 150.0, length=30);
σs = range(0.0, 80.0, length=30);

In [19]:
# Parameter values from the Cartesian product ρs × σs × {β}.
params = [(ρ, σ, p[3]) for ρ in ρs for σ in σs]

3600-element Vector{Tuple{Float64, Float64, Float64}}:
 (10.0, 0.0, 2.6666666666666665)
 (10.0, 1.3559322033898304, 2.6666666666666665)
 (10.0, 2.711864406779661, 2.6666666666666665)
 (10.0, 4.067796610169491, 2.6666666666666665)
 (10.0, 5.423728813559322, 2.6666666666666665)
 (10.0, 6.779661016949152, 2.6666666666666665)
 (10.0, 8.135593220338983, 2.6666666666666665)
 (10.0, 9.491525423728813, 2.6666666666666665)
 (10.0, 10.847457627118644, 2.6666666666666665)
 (10.0, 12.203389830508474, 2.6666666666666665)
 ⋮
 (150.0, 69.15254237288136, 2.6666666666666665)
 (150.0, 70.50847457627118, 2.6666666666666665)
 (150.0, 71.86440677966101, 2.6666666666666665)
 (150.0, 73.22033898305085, 2.6666666666666665)
 (150.0, 74.57627118644068, 2.6666666666666665)
 (150.0, 75.9322033898305, 2.6666666666666665)
 (150.0, 77.28813559322033, 2.6666666666666665)
 (150.0, 78.64406779661017, 2.6666666666666665)
 (150.0, 80.0, 2.6666666666666665)

In [20]:
function lorenz_static(u, p, _)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u
  
  # Define the Lorenz system equations.
  du = SVector{3}(
    σ * (y - x),
    x * (ρ - z) - y,
    x * y - β * z
  )

  # Return the derivatives.
  du
end

lorenz_static (generic function with 1 method)

In [21]:
function lorenz_static_jac(u, p, _)
  # Extract parameters.
  ρ, σ, β = p

  # Extract state variables.
  x, y, z = u

  # Define the Lorenz system Jacobian.
  jac = SMatrix{3, 3}(
     -σ,  σ,  0,
    ρ-z, -1, -x,
      y,  x, -β
  )

  # Return the Jacobian.
  jac
end

lorenz_static_jac (generic function with 1 method)

In [22]:
# Calculate Lyapunov spectra in parallel threads.

ds = CoupledODEs(lorenz_static, u0, p)
tands = TangentDynamicalSystem(ds; J = lorenz_static_jac)

# Preallocate the array for storing the Lyapunov spectra.
λs = zeros(length(params), 3)

# Since `DynamicalSystem`s are mutable, we need to copy to parallelize.
systems = [deepcopy(tands) for _ in 1:Threads.nthreads()-1]
pushfirst!(systems, tands)

Threads.@threads for i in eachindex(params)
    system = systems[Threads.threadid()]
    for param_idx in 1:3
        set_parameter!(system, param_idx, params[i][param_idx])
    end
    λs[i, :] .= lyapunovspectrum(system, Int(1e3); Ttr = 1e2)
end

# Display the calculated Lyapunov spectra.
λs

In [ ]:
# Store Leading Lyapunov exponents.
LLEs = [λs[i,1] for i in 1:size(λs)[1]]

# Store Lyapunov dimensions.
LDs = Float64[]
for i in 1:size(λs)[1]
  try
    push!(LDs, lyapunov_dimension(λs[i,:]))
  catch
    push!(LDs, NaN)
  end
end

In [ ]:
# Plot leading Lyapunov exponents (LLEs) as a heatmap.
fig = Figure()
ax = Axis(fig[1,1]; xlabel = L"\rho", ylabel = L"\sigma")
heatmap!(
  ax,
  ρs,
  σs,
  reshape(LLEs, (length(ρs), length(σs))),
  colormap = :thermal
)
Colorbar(fig[1, 2], colormap = :thermal, label = L"\lambda")
fig

In [17]:
# Plot Lyapunov dimensions (LDs) as a heatmap.
fig = Figure()
ax = Axis(fig[1,1]; xlabel = L"\rho", ylabel = L"\sigma")
heatmap!(
  ax,
  ρs,
  σs,
  reshape(LDs, (length(ρs), length(σs))),
  colormap = :thermal
)
Colorbar(fig[1, 2], colormap = :thermal, label = L"D_L")
